In [0]:
from pyspark import SparkContext
from pyspark.sql import *

In [0]:
df1 = spark.read.csv("/FileStore/tables/sharemarket.csv",inferSchema=True)

In [0]:
cols = "MARKET,SERIES,SYMBOL,SECURITY,PREV_CL_PR,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRICE,NET_TRDVAL,NET_TRDQTY,CORP_IND,TRADES,HI_52_WK,LO_52_WK".split(',')
df_sharemarket = df1.toDF(*(cols))
df_sharemarket.printSchema()
df_sharemarket.show(5)

root
 |-- MARKET: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SECURITY: string (nullable = true)
 |-- PREV_CL_PR: double (nullable = true)
 |-- OPEN_PRICE: double (nullable = true)
 |-- HIGH_PRICE: double (nullable = true)
 |-- LOW_PRICE: double (nullable = true)
 |-- CLOSE_PRICE: double (nullable = true)
 |-- NET_TRDVAL: double (nullable = true)
 |-- NET_TRDQTY: integer (nullable = true)
 |-- CORP_IND: string (nullable = true)
 |-- TRADES: integer (nullable = true)
 |-- HI_52_WK: double (nullable = true)
 |-- LO_52_WK: double (nullable = true)

+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+------+--------------------+----------+----------+----------+

In [0]:
df_sharemarket.createOrReplaceTempView("sharemarket")

In [0]:
# Q1

q1 = spark.sql("SELECT COUNT(DISTINCT(SERIES)) as num_of_series FROM sharemarket")
q1.show()

+-------------+
|num_of_series|
+-------------+
|           53|
+-------------+



In [0]:
# Q2

q2 = spark.sql("SELECT DISTINCT(SERIES) as series FROM sharemarket")
q2.show()

+------+
|series|
+------+
|    NL|
|    NK|
|    NJ|
|    NA|
|    N3|
|    NH|
|    NE|
|    NC|
|    ND|
|    N7|
|    N8|
|    N2|
|    N5|
|    N4|
|    N6|
|    NI|
|    N1|
|    NG|
|    N9|
|    NB|
+------+
only showing top 20 rows



In [0]:
# Q3

q3 = spark.sql("SELECT SERIES,SUM(CLOSE_PRICE) as sum_of_series FROM sharemarket GROUP BY SERIES")
q3.show()

+------+-------------+
|SERIES|sum_of_series|
+------+-------------+
|    YH|      1028.01|
|    NS|       1039.0|
|    NL|       3380.9|
|    NK|      1915.72|
|    Z4|       1022.1|
|    NJ|      4291.49|
|    NX|       295.07|
|    YS|       1135.0|
|    NW|       1089.0|
|    NA|     20059.57|
|    Z8|       1035.0|
|    YA|       300.98|
|    NP|       1291.0|
|    N3|      2087.38|
|    NH|      4705.37|
|    NE|     16846.83|
|    NC|      14382.9|
|    NQ|        315.0|
|    Z1|       1299.0|
|    NR|      1229.24|
+------+-------------+
only showing top 20 rows



In [0]:
# Q4 Display security,series with highest net trade value
q4= spark.sql("SELECT SERIES,SECURITY FROM sharemarket where NET_TRDVAL = (select max(NET_TRDVAL) from sharemarket)")
q4.show()


+------+--------------------+
|SERIES|            SECURITY|
+------+--------------------+
|    N6|8.75% TAX FREE TR...|
+------+--------------------+



In [0]:
#Q5.Display the series whose sum of all prices greater than the net trade value

q5= spark.sql("SELECT SERIES, (PREV_CL_PR+OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) as sum_all_prices, NET_TRDVAL from sharemarket where (PREV_CL_PR+OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE)>NET_TRDVAL")
q5.show()

+------+-----------------+----------+
|SERIES|   sum_all_prices|NET_TRDVAL|
+------+-----------------+----------+
|    N2|           6103.0|    3666.0|
|    N2|           5683.0|    4548.0|
|    N4|          5108.84|   2044.02|
|    N4|5159.200000000001|    4124.2|
|    N5|           4824.8|    1912.4|
|    N6|           6700.0|    1341.0|
|    N9|          26340.0|   21060.0|
|    NB|           4609.0|    1843.0|
|    Y7|           1555.2|    1500.0|
+------+-----------------+----------+



In [0]:
# 6.Display the series with highest net trade quantity

q6= spark.sql("SELECT SERIES,NET_TRDQTY FROM sharemarket WHERE NET_TRDQTY = (SELECT MAX(NET_TRDQTY) FROM sharemarket)")
q6.show()

+------+----------+
|SERIES|NET_TRDQTY|
+------+----------+
|    N3|    143810|
+------+----------+



In [0]:
# 7. Display the highest and lowest open price

q7= spark.sql("SELECT MAX(OPEN_PRICE) AS HIGHEST, MIN(OPEN_PRICE) AS LOWEST FROM sharemarket")
q7.show()

+-------+------+
|HIGHEST|LOWEST|
+-------+------+
|11450.0| 13.75|
+-------+------+



In [0]:
# 8.Query to display the series which have trades more than 80

q8= spark.sql("SELECT SERIES FROM sharemarket WHERE TRADES>80")
q8.show()

+------+
|SERIES|
+------+
|    N2|
|    N3|
|    N7|
+------+



In [0]:
# 9.Display the difference between the net trade value net trade quantity for each series

q9= spark.sql("SELECT SERIES, (NET_TRDVAL-NET_TRDQTY) AS DIFFERENCE_TRADE FROM sharemarket")
q9.show()

+------+----------------+
|SERIES|DIFFERENCE_TRADE|
+------+----------------+
|    N1|      3368851.44|
|    N1|         51319.0|
|    N1|      3857601.49|
|    N1|        113400.0|
|    N1|        108430.0|
|    N2|       656911.39|
|    N2|          3663.0|
|    N2|          4544.0|
|    N2|        422728.5|
|    N2|       124959.45|
|    N2|        464220.5|
|    N2|        128175.0|
|    N2|       526274.91|
|    N2|        238200.0|
|    N2|       1104894.5|
|    N3|      4089407.99|
|    N3|         78080.0|
|    N3|      1727457.92|
|    N4|         2042.02|
|    N4|         29616.6|
+------+----------------+
only showing top 20 rows

